In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchinfo import summary

import os
import time
import datetime as dt
from netCDF4 import Dataset as nc_Dataset
import pandas as pd
import numpy as np
import xarray as xr
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
import cartopy.feature as cfeature


from HRRR_URMA_Datasets_AllVars import *
from DefineModelAttributes import *
from SR_UNet_simple import *
from utils import *

In [2]:
MODEL_SAVE_DIR = f"/scratch/RTMA/alex.schein/CNN_Main/Trained_models"

In [3]:
for filename in [x for x in os.listdir(MODEL_SAVE_DIR) if ".pt" in x]:
    print(filename)
    x = DefineModelAttributes()
    x.set_model_attrs_from_savename(filename)
    #x.create_dataset()

BS256_NE1000_tD_pred(t2m)_targ(t2m).pt
BS256_NE1000_tD_pred(t2m-d2m-pressurf-u10m-v10m)_targ(t2m).pt
BS256_NE1000_tD_pred(t2m-d2m-pressurf-u10m)_targ(t2m).pt
BS256_NE1000_tD_pred(t2m-d2m-pressurf)_targ(t2m).pt
BS256_NE1000_tD_pred(t2m-d2m)_targ(t2m).pt


In [4]:
filename = f"BS256_NE1000_tD_pred(t2m-d2m)_targ(t2m).pt"
model_attrs = DefineModelAttributes()
model_attrs.set_model_attrs_from_savename(filename)
model_attrs.create_dataset()

Terrain normalization done for ['diff']
Loading predictor dataset for t2m (Years = 2021/22/23, months = 1 to 12, hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
Predictor dataset data loaded. Time taken = 5.5 sec
Normalizing over all times
Normalization done. Time taken = 1.6 sec
Loading predictor dataset for d2m (Years = 2021/22/23, months = 1 to 12, hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
Predictor dataset data loaded. Time taken = 4.9 sec
Normalizing over all times
Normalization done. Time taken = 1.6 sec
Loading target dataset for t2m (Years = 2021/22/23, months = 1 to 12, hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
Target dataset data loaded. Time taken = 5.3 sec
Normalizing over all times
Normalization done. Time taken = 1.6 sec
DATASET CONSTRUCTION DONE


In [5]:
model = SR_UNet_simple(n_channels_in=model_attrs.num_channels_in)
model.load_state_dict(torch.load(f"{MODEL_SAVE_DIR}/{filename}", weights_only=True))
device = "cuda:3"
model.to(device)

SR_UNet_simple(
  (first_conv): FirstConv(
    (conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), padding=same, bias=False)
      (1): LeakyReLU(negative_slope=0.1, inplace=True)
    )
  )
  (second_conv): ConvBlock(
    (conv): Sequential(
      (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
      (1): LeakyReLU(negative_slope=0.1, inplace=True)
    )
  )
  (enc_1): Encoder(
    (encoder): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): ConvBlock(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=same)
          (1): LeakyReLU(negative_slope=0.1, inplace=True)
        )
      )
    )
  )
  (enc_2): Encoder(
    (encoder): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): ConvBlock(
        (conv): Sequential(
          (0): Conv2d(128, 256, kernel_size=(5, 5), strid

In [6]:
pred, targ, model_output, dt_current = get_model_output_at_idx(model_attrs, model, pred_var="t2m", targ_var = "t2m", idx=0, is_unnormed=True)

In [ ]:
# Need to finish

# Try making a function to plot RMSE vs time - since it's spatial average, have 1 index per hour, so...